## Day 15 - HASH

**Part 1: Sum of the result of hash algorithm on comma separated string**

HASH:

- Determine the ASCII code for the current character of the string.
- Increase the current value by the ASCII code you just determined.
- Set the current value to itself multiplied by 17.
- Set the current value to the remainder of dividing itself by 256.

In [3]:
with open("./example.txt") as f:
    example_txt = f.read().strip()

with open("./input.txt") as f:
    input_txt = f.read().strip()

example_txt

'rn=1,cm-,qp=3,cm=2,qp-,pc=4,ot=9,ab=5,pc-,pc=6,ot=7'

In [9]:
def HASH(string: str) -> int:
    val = 0
    for ch in string:
        ascii_val = ord(ch)
        val += ascii_val
        val *= 17
        val = val % 256
    
    return val

HASH("HASH") # 52

52

In [12]:
def part1(input_txt: str) -> int:
    inputs = input_txt.split(",")

    total = 0
    for string in inputs:
        total += HASH(string)

    return total

assert part1(example_txt) == 1320
part1(input_txt)

510013

**Part 2: now there are boxes....**

For each input:

- initial characters are the label, and their HASH is the relevant box number
- `-` means go to the relevant box and remove the lens with the given label (if it exists there), then move any remaining lenses as far forward in the box as they can go without changing their order
- `=` is followed by a number, this tells you the *focal length* of the lens that needs to go into the relevant box. Can use label marker to mark the lens with the label given in the beginning of the step.
  - If there's already a lens in the box w same label, replace it with new one.
  - If there isn't a lens with that label, add the lens to the box immediately behind any lenses already there. 

Add up the *focusing power* of all the lenses. THe focusing power of a single lens is the result of multiplying together:
- One + the box number of the lens
- The slot number of the lens within the box (1 for the first lens, 2 for the second...)
- The focal length of the lens

e.g. `rn 1` in box 0 in the first slot:
 `1 (box 0) * 1 (first slot) * 1 (focal length) = 1`

In [14]:
example_txt

'rn=1,cm-,qp=3,cm=2,qp-,pc=4,ot=9,ab=5,pc-,pc=6,ot=7'

In [21]:
def part2(input_txt: str) -> int:
    inputs = input_txt.split(",")

    boxes = {
        i: [] for i in range(256)
    }
    
    for input in inputs:
        if "-" in input:
            removal_case = True
            label = input.split("-")[0]
        if "=" in input:
            removal_case = False
            label, val = input.split("=")
            val = int(val)
        
        box_number = HASH(label)
        
        if removal_case:
            # "-" case
            relevant_box = boxes[box_number]
            try:
                existing_label_idx = [lbl for lbl, _ in relevant_box].index(label)
                del relevant_box[existing_label_idx]
                boxes[box_number] = relevant_box  # don't think this line is necessary
            except ValueError:
                pass
        else:
            # "=" case
            relevant_box = boxes[box_number]
            try:
                existing_label_idx = [lbl for lbl, _ in relevant_box].index(label)
                relevant_box[existing_label_idx] = (label, val)
                boxes[box_number] = relevant_box
            except ValueError:
                relevant_box.append(
                    (label, val)
                )
                boxes[box_number] = relevant_box
    
    # Ok we have our boxes sorted
    # now to calculate total focal power

    total_focal_power = 0
    for box_number, lenses in boxes.items():
        for i, (label, focal_length) in enumerate(lenses):
            total_focal_power += (
                (1 + box_number) * (i + 1) * (focal_length)
            )
    return total_focal_power


assert part2(example_txt) == 145
part2(input_txt)

268497